In [5]:
import pandas as pd
import pickle

In [6]:
games_data=pd.read_csv('C:/Users/19928/Desktop/Projects/Project-1/Code/game.csv')
games_data.head()

,AppID,Name,About the game,Categories,Genres,Tags
0,2525270,The House,' The House ' is a short psychological horror ...,"Single-player,Steam Workshop","Adventure,Casual,Indie","Adventure,Casual,Action-Adventure,Walking Simu..."
1,2414640,Bite Size Terrors: Erobos Heaven,Bite Size Terrors: Are Short Experimental Horr...,"Single-player,Steam Achievements","Adventure,Indie","Horror,Adventure,Atmospheric,Psychological Hor..."
2,2486670,TD Tower Defense,Td Tower Defense is a fun tower defense shoote...,Single-player,"Action,Casual","Tower Defense,Casual,Sci-fi,Strategy,Combat,Re..."
3,2304650,The Scrap,The Scrap is an independent third-person shoot...,"Single-player,Steam Achievements","Action,Adventure,Indie","Action,Shooter,Third-Person Shooter,3D,Third P..."
4,2519670,Wind Love,Gameplay Wind Love - is a Japanese-style visua...,"Single-player,Steam Achievements,Captions avai...","Indie,Simulation","Hentai,Adventure,Dating Sim,Casual,Choices Mat..."


In [8]:
games_data=games_data[['AppID','Name','Genres']]

In [9]:
#Natural language processing
#通过Genres来推荐游戏
new_games_data=pd.DataFrame(games_data)
#字符串按空格进行分割
new_games_data['Genres']=new_games_data['Genres'].apply(lambda x:x.split())
#移除Genres中的空格
new_games_data['Genres']=new_games_data['Genres'].apply(lambda x:[i.replace(" ","") for i in x])
#形成新Dataframe
new_df=new_games_data[['AppID','Name','Genres']]
#小写并通过空格连接
new_df['Genres']=new_df['Genres'].apply(lambda x:" ".join(x))
new_df['Genres']=new_df['Genres'].apply(lambda x:x.lower())
#词干提取
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)
new_df['Genres']=new_df['Genres'].apply(stem)

In [11]:
#计数向量,每个单词在每一个Genres字段是否出现，是：1，否;0
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=100,stop_words='english')
vectors=cv.fit_transform(new_df['Genres']).toarray()
print(vectors.shape)
#Genres共54个

(18560, 54)


In [54]:
from sklearn.metrics.pairwise import euclidean_distances
def recommend_genre(genre):
    #标准化输入
    genre = cv.transform([genre]).toarray()
    #计算距离
    distances=[]
    for i in range(18560):
        a=vectors[i,:]
        a=a.reshape(1, -1)
        distance = euclidean_distances(genre,a)
        distances.append((distance, i))
    #计算最小的五个距离
    distances = sorted(distances, key=lambda x: x[0]) 
    min_distances = distances[:5]
    #相应index
    min_indexes = [idx for _, idx in min_distances]
    #print出对应游戏名
    for idx in min_indexes:
        movie_name = new_df.iloc[idx, 1] 
        print(movie_name)

In [58]:
save_path = 'C:/Users/19928/Desktop/Projects/Project-1/Code/recommend_genre.pkl'
pickle.dump(recommend_genre, open(save_path, 'wb'))
save_path = 'C:/Users/19928/Desktop/Projects/Project-1/Code/vectors.pkl'
pickle.dump(vectors, open(save_path, 'wb'))
save_path = 'C:/Users/19928/Desktop/Projects/Project-1/Code/cv.pkl'
pickle.dump(cv, open(save_path, 'wb'))

1


In [55]:
recommend_genre("action,casual")

Super Space Club
Nebula's Descent
Sclash
KAGITORI -BIRD IN THE CAGE HIDING THE KEY-
PLONG
